In [13]:
#https://www.youtube.com/watch?v=OtCq4NGvTaA
import pandas as pd
import numpy as np
import statsmodels as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

import pprint
from pprint import pprint

In [14]:
df=pd.read_csv('creditcard.csv')
df = df.rename(columns={'Class': 'Fraud'})

In [15]:
df['Fraud'] = df['Fraud'].astype(int)

X = df.drop(['Fraud'], axis = 1)
Y = df["Fraud"]

xData = X.values
yData = Y.values

In [16]:
xTrain, xTest, yTrain, yTest = train_test_split(
        xData, yData, test_size = 0.2, random_state = 42)
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

#This is where we started changing things
#pipe = Pipeline([('standardScaler', StandardScaler()), ('quantiletransformer', QuantileTransformer()), 
                 #('Random Forest', RFR(n_estimators=10, max_depth=7))])

In [17]:
RandomForestClassifier().get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [18]:
n_estimators_list = [130,160,190]
criterion_list = ['gini']
max_depth_list = [35,55]
min_samples_split_list = [0.001, 0.005]
min_samples_leaf_list = [0.001, 0.005]
max_features_list = ['log2']

params_grid = {
    'n_estimators': n_estimators_list,
    'criterion': criterion_list,
    'max_depth': max_depth_list,
    'min_samples_split': min_samples_split_list,
    'min_samples_leaf': min_samples_leaf_list,
    'max_features': max_features_list
}

num_combinations = 1
for k in params_grid.keys(): num_combinations *= len(params_grid[k])

print('Number of combinations = ', num_combinations)
params_grid

Number of combinations =  24


{'n_estimators': [130, 160, 190],
 'criterion': ['gini'],
 'max_depth': [35, 55],
 'min_samples_split': [0.001, 0.005],
 'min_samples_leaf': [0.001, 0.005],
 'max_features': ['log2']}

In [19]:
def my_roc_auc_score(model, xTrain, yTrain): return metrics.roc_auc_score(yTrain, model.predict(xTrain))

model_rf = GridSearchCV(estimator=RandomForestClassifier(class_weight='balanced'),
                        param_grid=params_grid,
                        cv=3,
                        scoring=my_roc_auc_score,
                        return_train_score=True,
                        verbose=4)

model_rf.fit(xTrain, yTrain)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV 1/3] END criterion=gini, max_depth=35, max_features=log2, min_samples_leaf=0.001, min_samples_split=0.001, n_estimators=130;, score=(train=0.992, test=0.916) total time= 1.3min
[CV 2/3] END criterion=gini, max_depth=35, max_features=log2, min_samples_leaf=0.001, min_samples_split=0.001, n_estimators=130;, score=(train=0.992, test=0.912) total time= 1.3min
[CV 3/3] END criterion=gini, max_depth=35, max_features=log2, min_samples_leaf=0.001, min_samples_split=0.001, n_estimators=130;, score=(train=0.992, test=0.942) total time= 1.3min
[CV 1/3] END criterion=gini, max_depth=35, max_features=log2, min_samples_leaf=0.001, min_samples_split=0.001, n_estimators=160;, score=(train=0.998, test=0.916) total time= 1.9min
[CV 2/3] END criterion=gini, max_depth=35, max_features=log2, min_samples_leaf=0.001, min_samples_split=0.001, n_estimators=160;, score=(train=0.996, test=0.908) total time= 1.9min
[CV 3/3] END criterion=gini, max_d

[CV 1/3] END criterion=gini, max_depth=55, max_features=log2, min_samples_leaf=0.001, min_samples_split=0.005, n_estimators=130;, score=(train=0.977, test=0.919) total time= 1.3min
[CV 2/3] END criterion=gini, max_depth=55, max_features=log2, min_samples_leaf=0.001, min_samples_split=0.005, n_estimators=130;, score=(train=0.976, test=0.919) total time= 1.3min
[CV 3/3] END criterion=gini, max_depth=55, max_features=log2, min_samples_leaf=0.001, min_samples_split=0.005, n_estimators=130;, score=(train=0.979, test=0.945) total time= 1.2min
[CV 1/3] END criterion=gini, max_depth=55, max_features=log2, min_samples_leaf=0.001, min_samples_split=0.005, n_estimators=160;, score=(train=0.979, test=0.918) total time= 1.5min
[CV 2/3] END criterion=gini, max_depth=55, max_features=log2, min_samples_leaf=0.001, min_samples_split=0.005, n_estimators=160;, score=(train=0.976, test=0.919) total time= 1.6min
[CV 3/3] END criterion=gini, max_depth=55, max_features=log2, min_samples_leaf=0.001, min_sampl

GridSearchCV(cv=3, estimator=RandomForestClassifier(class_weight='balanced'),
             param_grid={'criterion': ['gini'], 'max_depth': [35, 55],
                         'max_features': ['log2'],
                         'min_samples_leaf': [0.001, 0.005],
                         'min_samples_split': [0.001, 0.005],
                         'n_estimators': [130, 160, 190]},
             return_train_score=True,
             scoring=<function my_roc_auc_score at 0x11df1eae8>, verbose=4)

In [22]:
df_cv_results = pd.DataFrame(model_rf.cv_results_)
df_cv_results = df_cv_results[['rank_test_score','mean_test_score','mean_train_score',
                           'param_n_estimators', 'param_min_samples_split','param_min_samples_leaf',
                           'param_max_features', 'param_max_depth','param_criterion']]
df_cv_results.sort_values('rank_test_score', inplace=True)
df_cv_results

,rank_test_score,mean_test_score,mean_train_score,param_n_estimators,param_min_samples_split,param_min_samples_leaf,param_max_features,param_max_depth,param_criterion
8,1,0.930062,0.957228,190,0.001,0.005,log2,35,gini
6,2,0.929069,0.958206,130,0.001,0.005,log2,35,gini
10,3,0.929003,0.955606,160,0.005,0.005,log2,35,gini
22,4,0.928867,0.955456,160,0.005,0.005,log2,55,gini
9,5,0.928808,0.957938,130,0.005,0.005,log2,35,gini
20,6,0.928572,0.956451,190,0.001,0.005,log2,55,gini
23,7,0.927927,0.955735,190,0.005,0.005,log2,55,gini
11,8,0.927620,0.959857,190,0.005,0.005,log2,35,gini
19,9,0.927613,0.957339,160,0.001,0.005,log2,55,gini
5,10,0.927562,0.979018,190,0.005,0.001,log2,35,gini


In [32]:
#output_dir = '/kaggle/working/'
#df_cv_results.to_csv(output_dir + '/grid_search.csv', index=False)

output_dir= '/Users/skamboj2022/Downloads/ACM Projects'
df_cv_results.to_csv(output_dir + '/grid_search.csv', index=False)


In [33]:
model_rf.best_params_

{'criterion': 'gini',
 'max_depth': 35,
 'max_features': 'log2',
 'min_samples_leaf': 0.005,
 'min_samples_split': 0.001,
 'n_estimators': 190}

Create model based off of best parameters

In [34]:
model_rf_fin = RandomForestClassifier(class_weight='balanced',
                                      criterion='gini',
                                      max_depth=35,
                                      max_features='log2',
                                      min_samples_leaf=0.005,
                                      min_samples_split=0.001,
                                      n_estimators=190)
model_rf_fin.fit(xTrain, yTrain)

RandomForestClassifier(class_weight='balanced', max_depth=35,
                       max_features='log2', min_samples_leaf=0.005,
                       min_samples_split=0.001, n_estimators=190)

In [35]:
print('AUC Score = {:.4f}'.format(metrics.roc_auc_score(yTrain, model_rf_fin.predict(xTrain))))

AUC Score = 0.9508
